In [16]:
import os
os.environ["WANDB_DISABLED"] = "true"

import sys
sys.path.append('..')

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import analyze_utils

checkpoint_folder = "/home/jxm3/research/retrieval/inversion/saves/98b1418d38c3f9333b17ab20bff06ff9/"
trainer = analyze_utils.load_trainer(checkpoint_folder)

Loading trainer for analysis – setting --do_eval=1
loading alias dpr_nq__msl32_beta from /home/jxm3/research/retrieval/inversion/saves/db66b9c01b644541fedbdcc59c53a285/ebb31d91810c4b62d2b55b5382e8c7ea...
Set train_args.dataloader_num_workers = 4


/home/jxm3/.conda/envs/torch/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[nltk_data] Downloading package wordnet to /home/jxm3/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/jxm3/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/jxm3/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


================ Begin trainer sanity check ================
	Input to encode -> Twas brillig, and the slithy toves, Did gyre and gimble in the wabe, All mimsy were the borogoves, And the mome raths outgrabe.
generation_kwargs: {'early_stopping': False, 'num_beams': 1, 'do_sample': False, 'no_repeat_ngram_size': 3, 'max_length': 58}
	Decoded output -> The mlbies wase wyst bograge; And the sliths and toms wre
================ End trainer sanity check ================


[nltk_data] Downloading package wordnet to /home/jxm3/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/jxm3/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/jxm3/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [4]:
trainer.eval_dataset.keys()

dict_keys(['ag_news', 'xsum_doc', 'xsum_summ', 'wikibio', 'nq'])

In [64]:
trainer.args.per_device_eval_batch_size = 8
eval_batch = next(iter(trainer.get_eval_dataloader(
    eval_dataset=trainer.eval_dataset["nq"]
)))
eval_batch = {k: v[0:1, ...] for k,v in eval_batch.items()} # just one guy
eval_batch = {k: v.to(trainer.args.device) for k,v in eval_batch.items()}

trainer.inversion_trainer.contrastive_generation_alpha = 0.0
trainer.inversion_trainer.contrastive_generation_gamma = 0.1

gen_kwargs = { "max_length": 32,   "do_sample": False, }
trainer.inversion_trainer.generate_contrastive(inputs=eval_batch, generation_kwargs=gen_kwargs)

generation_kwargs: {'max_length': 32, 'early_stopping': True, 'num_beams': 1, 'do_sample': False}
ContrastiveLogitsProcessor alpha=0.0 gamma=0.1
generation_kwargs: {'max_length': 32, 'do_sample': False, 'logits_processor': [<models.logits_processors.ContrastiveLogitsProcessor object at 0x7fb99950e020>], 'renormalize_logits': False}


tensor([[    0,    12,    24,    13,     8, 21995,    13,  9806,     7,     6,
            84,    33,  4352,  3050,    11,    73,    17,    76,   235,  8283,
            16,  1848,     5,     3, 13099,     6,     8,  2629, 10471,    13,
             8, 21995]], device='cuda:0')

In [32]:
trainer.inversion_trainer.generate(inputs=eval_batch, generation_kwargs={
    "max_length": 32,
    "do_sample": False,
})

generation_kwargs: {'max_length': 32, 'do_sample': False}


tensor([[    0,    12,     8,  1848,    13,     8, 21995,     6,    84,    33,
          4352,  3050,    11,    73,    17,    76,    17, 13281,    16,     8,
           495,    13,  9806,  2629, 10471,     5,     3,  8739,     8,  2629,
         10471,     6]], device='cuda:0')

In [65]:
trainer.args.per_device_eval_batch_size = 8
eval_batch = next(iter(trainer.get_eval_dataloader(
    eval_dataset=trainer.eval_dataset["nq"]
)))

# eval_batch = {k: v[0:1, ...] for k,v in eval_batch.items()} # just one guy
eval_batch = {k: v.to(trainer.args.device) for k,v in eval_batch.items()}

gen_kwargs = {"max_length": 32,   "do_sample": False, }
trainer.inversion_trainer.contrastive_generation_alpha = 0.0
trainer.inversion_trainer.contrastive_generation_gamma = 1.0
b1 = trainer.inversion_trainer.generate_contrastive(inputs=eval_batch, generation_kwargs=gen_kwargs)
b2 = trainer.inversion_trainer.model.generate(inputs=eval_batch, generation_kwargs=gen_kwargs)

generation_kwargs: {'max_length': 32, 'early_stopping': True, 'num_beams': 1, 'do_sample': False}
ContrastiveLogitsProcessor alpha=0.0 gamma=1.0
generation_kwargs: {'max_length': 32, 'do_sample': False, 'logits_processor': [<models.logits_processors.ContrastiveLogitsProcessor object at 0x7fb99950d9c0>], 'renormalize_logits': False}
generation_kwargs: {'max_length': 32, 'do_sample': False, 'logits_processor': [<models.logits_processors.ContrastiveLogitsProcessor object at 0x7fb99950d9c0>], 'renormalize_logits': False}


In [66]:
(b1 == b2).all()

tensor(True, device='cuda:0')

In [52]:
trainer.inversion_trainer.generate_contrastive(inputs=eval_batch, generation_kwargs={"max_length": 32,   "do_sample": False, })

generation_kwargs: {'max_length': 32, 'early_stopping': True, 'num_beams': 1, 'do_sample': False}
generation_kwargs: {'max_length': 32, 'do_sample': False, 'logits_processor': [<models.logits_processors.ContrastiveLogitsProcessor object at 0x7fb876c50550>], 'renormalize_logits': True}


tensor([[    0,    12,     8,  1848,    13,     8, 21995,     6,    84,    33,
          4352,  3050,    11,    73,    17,    76,    17, 13281,    16,     8,
           495,    13,  9806,  2629, 10471,     5,     3,  8739,     8,  2629,
         10471,     6],
        [    0,   928,    45,     8,  5193,   216,    51,    23,  9475,    12,
          1369,     8,  4272, 11548, 16010,  2045,  9365,     6,    11,    47,
            92,   294,    13,     8,  3746,  7110,  8799,   372,     6,    84,
           751,  2045],
        [    0,     8,   337,  2166,    38,  1076,     6,    11,   489,  2712,
          4686,    24,    79,   225,    36,  5046,    45,  9192,   257,     5,
             3,  7264,     8,   119, 13914,     6, 16998,  2765,    33,   150,
          1200,  8116],
        [    0,     3,  9465,   189,    35,     6,  7488,     3,  9465,   189,
            35,    19,     3,     9,   422,  1511,  1069,   344,     8,  2473,
          2067,  6310,    11,     3,  9465,  5437,    30,  

## eval

In [59]:
trainer.inversion_trainer.evaluate(
    eval_dataset=trainer.eval_dataset["nq"]
)

to the character of the skull, which are relatively smooth and untutuous in the case of infant sutures. Unlike the sutures,
to the character of the sutures of the skull which, like those of the infant skull, are relatively smooth and untortuous. In fact



individual from the Southern Hemisphere to win the Winter Olympic relay gold medal, and was also part of the Australian Short Track team, which won gold
individual from the Southern Hemisphere, to win a Winter Olympic gold medal and was also part of the short track relay team that won Australia'



the same rights as men, and 75% agreed that they should be protected from discrimination. Among the other 15%, gay workers are no longer exp
the same rights as straight people, while 15% disagreed. Additionally, 69% agreed that they should be protected from workplace discrimination. 13% of H


{'eval_loss': 1.0522613525390625,
 'eval_accuracy': 0.7386067708333334,
 'eval_bleu_score': 28.897897788124407,
 'eval_meteor_score': 0.5641452367467923,
 'eval_rouge_score': 0.6308596689442678,
 'eval_bert_score': 28.897897788124407,
 'eval_emb_cos_sim': 0.9019443988800049,
 'eval_perplexity': 2.8641205867926014,
 'eval_runtime': 19.8886,
 'eval_samples_per_second': 25.14,
 'eval_steps_per_second': 0.201}

In [60]:
trainer.inversion_trainer.gen_kwargs

{'early_stopping': False,
 'num_beams': 1,
 'do_sample': False,
 'no_repeat_ngram_size': 3}

In [133]:
trainer.inversion_trainer.generation_strategy = "none"
trainer.inversion_trainer.gen_kwargs["no_repeat_ngram_size"] = 100000000
trainer.inversion_trainer.evaluate(
    eval_dataset=trainer.eval_dataset["nq"]
)

generating from val:   0%|                                                                                                                                  | 0/3 [00:00<?, ?it/s]

generation_kwargs: {'early_stopping': False, 'num_beams': 1, 'do_sample': False, 'no_repeat_ngram_size': 100000000, 'min_length': 32, 'max_length': 32}


generating from val:  33%|████████████████████████████████████████▋                                                                                 | 1/3 [00:00<00:01,  1.13it/s]

generation_kwargs: {'early_stopping': False, 'num_beams': 1, 'do_sample': False, 'no_repeat_ngram_size': 100000000, 'min_length': 32, 'max_length': 32}


generating from val:  67%|█████████████████████████████████████████████████████████████████████████████████▎                                        | 2/3 [00:01<00:00,  1.39it/s]

generation_kwargs: {'early_stopping': False, 'num_beams': 1, 'do_sample': False, 'no_repeat_ngram_size': 100000000, 'min_length': 32, 'max_length': 32}


to the character of the skull, which are relatively smooth and untutuous in the case of infant sutures. Unlike the sutures,
to the character of the sutures of the skull which, like those of the infant skull, are relatively smooth and untortuous. In fact



individual from the Southern Hemisphere to win the Winter Olympic relay gold medal, and was also part of the Australian Short Track team, which won gold
individual from the Southern Hemisphere, to win a Winter Olympic gold medal and was also part of the short track relay team that won Australia'



the same rights as men, and 75% agreed that they should be protected from discrimination. Among the other 15%, gay workers are no longer exp
the same rights as straight people, while 15% disagreed. Additionally, 69% agreed that they should be protected from workplace discrimination. 13% of H


{'eval_loss': 1.0522613525390625,
 'eval_accuracy': 0.7386067708333334,
 'eval_bleu_score': 31.876985502984844,
 'eval_meteor_score': 0.5792206976897826,
 'eval_rouge_score': 0.6414878805144706,
 'eval_bert_score': 0.9177108282844225,
 'eval_emb_cos_sim': 0.9062262773513794,
 'eval_perplexity': 2.8641205867926014,
 'eval_runtime': 4.999,
 'eval_samples_per_second': 100.021,
 'eval_steps_per_second': 0.8}

In [151]:
trainer.inversion_trainer.generation_strategy = "contrastive"
trainer.inversion_trainer.evaluate(
    eval_dataset=trainer.eval_dataset["nq"]
)

generating from val:   0%|                                                                                                                                  | 0/3 [00:00<?, ?it/s]

contrastive alpha = 0.0
generation_kwargs: {'max_length': 32, 'early_stopping': True, 'num_beams': 1, 'do_sample': False}


generating from val:  33%|████████████████████████████████████████▋                                                                                 | 1/3 [00:04<00:09,  4.82s/it]

contrastive alpha = 0.0
generation_kwargs: {'max_length': 32, 'early_stopping': True, 'num_beams': 1, 'do_sample': False}


generating from val:  67%|█████████████████████████████████████████████████████████████████████████████████▎                                        | 2/3 [00:09<00:04,  4.66s/it]

contrastive alpha = 0.0
generation_kwargs: {'max_length': 32, 'early_stopping': True, 'num_beams': 1, 'do_sample': False}


to the character of the skull, which are relatively smooth and untutuous in the case of infant sutures. Unlike the sutures,
to the character of the sutures of the skull which, like those of the infant skull, are relatively smooth and untortuous. In fact



individual from the Southern Hemisphere to win the Winter Olympic relay gold medal, and was also part of the Australian Short Track team, which won gold
individual from the Southern Hemisphere, to win a Winter Olympic gold medal and was also part of the short track relay team that won Australia'



the same rights as men, and 75% agreed that they should be protected from discrimination. Among the other 15%, non-Hispanic workplace
the same rights as straight people, while 15% disagreed. Additionally, 69% agreed that they should be protected from workplace discrimination. 13% of H


{'eval_loss': 1.0522613525390625,
 'eval_accuracy': 0.7386067708333334,
 'eval_bleu_score': 32.81495024580276,
 'eval_meteor_score': 0.5880659124678774,
 'eval_rouge_score': 0.650595111087725,
 'eval_bert_score': 0.9192223938492438,
 'eval_emb_cos_sim': 0.9107081890106201,
 'eval_perplexity': 2.8641205867926014,
 'eval_runtime': 16.9251,
 'eval_samples_per_second': 29.542,
 'eval_steps_per_second': 0.236}

In [69]:
trainer.inversion_trainer.generation_strategy = "contrastive"
trainer.inversion_trainer.gen_kwargs["no_repeat_ngram_size"] = 100000000
trainer.inversion_trainer.contrastive_generation_alpha = 0.0
trainer.inversion_trainer.contrastive_generation_gamma = 0.1
trainer.inversion_trainer.evaluate(
    eval_dataset=trainer.eval_dataset["nq"]
)

generating from val:   0%|                                                                                                                                  | 0/3 [00:00<?, ?it/s]

generation_kwargs: {'max_length': 32, 'early_stopping': True, 'num_beams': 1, 'do_sample': False}
ContrastiveLogitsProcessor alpha=0.0 gamma=0.1
generation_kwargs: {'early_stopping': False, 'num_beams': 1, 'do_sample': False, 'no_repeat_ngram_size': 100000000, 'min_length': 32, 'max_length': 32, 'logits_processor': [<models.logits_processors.ContrastiveLogitsProcessor object at 0x7fb876b796c0>], 'renormalize_logits': False}


generating from val:  33%|████████████████████████████████████████▋                                                                                 | 1/3 [00:03<00:06,  3.38s/it]

generation_kwargs: {'max_length': 32, 'early_stopping': True, 'num_beams': 1, 'do_sample': False}
ContrastiveLogitsProcessor alpha=0.0 gamma=0.1
generation_kwargs: {'early_stopping': False, 'num_beams': 1, 'do_sample': False, 'no_repeat_ngram_size': 100000000, 'min_length': 32, 'max_length': 32, 'logits_processor': [<models.logits_processors.ContrastiveLogitsProcessor object at 0x7fb876eab0d0>], 'renormalize_logits': False}


generating from val:  67%|█████████████████████████████████████████████████████████████████████████████████▎                                        | 2/3 [00:06<00:03,  3.22s/it]

generation_kwargs: {'max_length': 32, 'early_stopping': True, 'num_beams': 1, 'do_sample': False}
ContrastiveLogitsProcessor alpha=0.0 gamma=0.1
generation_kwargs: {'early_stopping': False, 'num_beams': 1, 'do_sample': False, 'no_repeat_ngram_size': 100000000, 'min_length': 32, 'max_length': 32, 'logits_processor': [<models.logits_processors.ContrastiveLogitsProcessor object at 0x7fb876bc5510>], 'renormalize_logits': False}


to that of the skull of infants, which are relatively smooth and untutorous in character. Nevertheless, the sutures of the skull
to the character of the sutures of the skull which, like those of the infant skull, are relatively smooth and untortuous. In fact



individual hemisphere athlete from Australia to win the Winter Olympic relay, and was part of the Southern Hemisphere team, which won the short
individual from the Southern Hemisphere, to win a Winter Olympic gold medal and was also part of the short track relay team that won Australia'



the same rights as straight people, and 7% disagreed. Moreover, 35% agreed that they should be protected from workplace discrimination. 75%
the same rights as straight people, while 15% disagreed. Additionally, 69% agreed that they should be protected from workplace discrimination. 13% of H


{'eval_loss': 1.0522613525390625,
 'eval_accuracy': 0.7386067708333334,
 'eval_bleu_score': 33.98037368971617,
 'eval_meteor_score': 0.601873441349148,
 'eval_rouge_score': 0.6646144531814782,
 'eval_bert_score': 0.9221128008017937,
 'eval_emb_cos_sim': 0.9135487079620361,
 'eval_perplexity': 2.8641205867926014,
 'eval_runtime': 12.4936,
 'eval_samples_per_second': 40.021,
 'eval_steps_per_second': 0.32}